In [34]:
from collections import defaultdict

import tqdm.notebook as tqdm

In [197]:
input_file = "24_input.txt"
#input_file = "24_sample.txt"

In [198]:
with open(input_file) as f:
    valleymap = [line.rstrip().strip("#") for line in f]
valleymap = valleymap[1:-1]

In [199]:
class Cell:
    def __init__(self, y, x, h, w):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.freex = {k: True for k in range(w)}
        self.freey = {k: True for k in range(h)}
    
    def is_free(self, time):
        return self.freex[time % self.w] and self.freey[time % self.h]
            

class Valley:
    def __init__(self, t0map):
        self.h = len(t0map)
        self.w = len(t0map[0])
        self.map = [[Cell(y, x, self.h, self.w) for x in range(self.w)] for y in range(self.h)]
        self.start = (-1, 0)
        self.end = (self.h, self.w-1)
        
        for y, row in enumerate(t0map):
            for x, c in enumerate(row):
                if c == ">":
                    for dt in range(self.w):
                        self.map[y][(x + dt) % self.w].freex[dt] = False
                elif c == "<":
                    for dt in range(self.w):
                        self.map[y][(x - dt) % self.w].freex[dt] = False
                elif c == "^":
                    for dt in range(self.h):
                        self.map[(y - dt) % self.h][x].freey[dt] = False
                elif c == "v":
                    for dt in range(self.h):
                        self.map[(y + dt) % self.h][x].freey[dt] = False
    
    def neighbours(self, y, x):
        if y == -1:
            return [(0, 0)]#, (y, x)]
        if y == self.h:
            return [(self.h-1, self.w-1)]#, (y, x)]
        n = [(y, x)]
        if y > 0:
            n.append((y - 1, x))
        if y < self.h - 1:
            n.append((y + 1, x))
        if x > 0:
            n.append((y, x - 1))
        if x < self.w - 1:
            n.append((y, x + 1))
        return n
    
    def find_path(self):
        q = [[self.start]]
        length = 0
        c = 0
        with tqdm.tqdm() as progbar:
            while q:
                path = q.pop(0)
                c += 1
                if c % TQUPD == 0:
                    progbar.update(TQUPD)
                    progbar.set_description(f"l={length}, q={len(q)}")
                if len(path) > length:
                    length = len(path)
                    progbar.set_description(f"l={length}, q={len(q)}")
                for y, x in self.neighbours(*path[-1]):
                    if self.map[y][x].is_free(time=len(path)+1):
                        if (y, x) == (self.h-1, self.w-1):
                            return len(path) + 2
                        else:
                            q.append(path + [(y, x)])
            return -1

    def print_available(self, time=0, chars=' .'):
        for y, row in enumerate(self.map):
            print(''.join(chars[cell.is_free(time)] for cell in row))
    
    
    def shortest_path(self, start_time=0):
        paths = defaultdict(set)
        l = start_time
        # while not(self.map[0][0].is_free(time=l+1)):
        #     l += 1
        paths[l].add(self.start)
        while True:
            for current in paths[l]:
                if current[0] == -1:
                    paths[l+1].add(current)
                for y, x in self.neighbours(*current):   
                    if self.map[y][x].is_free(time=(l+1)):
                        if (y, x) == (self.h-1, self.w-1):
                            return l + 2 - start_time
                        paths[l+1].add((y, x))
            l += 1
    
    
    def reverse_path(self, start_time=0):
        paths = defaultdict(set)
        l = start_time
        while not(self.map[-1][-1].is_free(time=l+1)):
            l += 1
        paths[l].add(self.end)
        while True:
            if not(paths[l]):
                raise ValueError(f"no path at {l}")
            for current in paths[l]:
                if current[0] == self.h:
                    paths[l+1].add(current)
                for y, x in self.neighbours(*current):
                    if self.map[y][x].is_free(time=l+1):
                        if (y, x) == (0, 0):
                            return l + 2 - start_time
                        paths[l+1].add((y,x))
            l += 1
    
    def part2(self):
        t1 = self.shortest_path()
        t2 = self.reverse_path(t1)
        t3 = self.shortest_path(t1+t2)
        return t1+t2+t3, (t1, t2, t3)

TQUPD = 10**4

In [200]:
valley = Valley(valleymap)

In [201]:
valley.shortest_path(start_time=41)

254

In [202]:
valley.part2()

(816, (292, 241, 283))